In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/freshness2/data/fruits_split/test/Apple(5-10)/frame194.jpg
/kaggle/input/freshness2/data/fruits_split/test/Apple(5-10)/frame400 - Copy.jpg
/kaggle/input/freshness2/data/fruits_split/test/Apple(5-10)/frame213.jpg
/kaggle/input/freshness2/data/fruits_split/test/Apple(5-10)/frame154.jpg
/kaggle/input/freshness2/data/fruits_split/test/Apple(5-10)/frame224.jpg
/kaggle/input/freshness2/data/fruits_split/test/Apple(5-10)/frame260.jpg
/kaggle/input/freshness2/data/fruits_split/test/Apple(5-10)/frame650.jpg
/kaggle/input/freshness2/data/fruits_split/test/Apple(5-10)/frame324.jpg
/kaggle/input/freshness2/data/fruits_split/test/Apple(5-10)/frame290.jpg
/kaggle/input/freshness2/data/fruits_split/test/Apple(5-10)/frame710.jpg
/kaggle/input/freshness2/data/fruits_split/test/Apple(5-10)/frame800.jpg
/kaggle/input/freshness2/data/fruits_split/test/Apple(5-10)/frame320.jpg
/kaggle/input/freshness2/data/fruits_split/test/Apple(5-10)/frame426.jpg
/kaggle/input/freshness2/data/fruits_split/t

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, models, transforms
import time

# Set device (GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Directories for train and test data
train_dir = '/kaggle/input/freshness1/data/fruits_split/train'
test_dir = '/kaggle/input/freshness1/data/fruits_split/test'

# Define data transformations for train and test sets
train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # Pretrained model normalization
])

test_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Load datasets
train_dataset = datasets.ImageFolder(train_dir, transform=train_transforms)
test_dataset = datasets.ImageFolder(test_dir, transform=test_transforms)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Load the pre-trained VGG16 model
model = models.vgg16(pretrained=True)

# Freeze the feature extraction layers
for param in model.features.parameters():
    param.requires_grad = False

# Modify the classifier part to match our number of classes (e.g., 13 for fruit freshness)
num_classes = len(train_dataset.classes)
model.classifier[6] = nn.Linear(model.classifier[6].in_features, num_classes)

# Move model to the appropriate device
model = model.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.classifier.parameters(), lr=0.001)

# Training function
def train_model(model, criterion, optimizer, train_loader, num_epochs=25):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        correct = 0
        total = 0
        start_time = time.time()
        
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            
            # Zero the parameter gradients
            optimizer.zero_grad()
            
            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            # Backward pass and optimization
            loss.backward()
            optimizer.step()
            
            # Track the loss and accuracy
            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        
        epoch_loss = running_loss / len(train_loader)
        epoch_acc = 100 * correct / total
        elapsed_time = time.time() - start_time
        
        print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.2f}%, Time: {elapsed_time:.2f}s')

# Evaluation function
def evaluate_model(model, test_loader):
    model.eval()  # Set model to evaluation mode
    correct = 0
    total = 0
    with torch.no_grad():  # No need to track gradients for evaluation
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    accuracy = 100 * correct / total
    print(f'Test Accuracy: {accuracy:.2f}%')

# Train the model
train_model(model, criterion, optimizer, train_loader, num_epochs=10)

# Evaluate the model on the test set
evaluate_model(model, test_loader)


/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:02<00:00, 221MB/s]


Epoch 1/10, Loss: 2.1781, Accuracy: 52.66%, Time: 29.86s
Epoch 2/10, Loss: 1.5853, Accuracy: 66.57%, Time: 20.29s
Epoch 3/10, Loss: 1.5483, Accuracy: 69.60%, Time: 20.55s
Epoch 4/10, Loss: 1.7326, Accuracy: 73.67%, Time: 20.87s
Epoch 5/10, Loss: 2.0370, Accuracy: 74.19%, Time: 20.42s
Epoch 6/10, Loss: 2.0906, Accuracy: 73.96%, Time: 20.57s
Epoch 7/10, Loss: 1.8877, Accuracy: 77.74%, Time: 20.86s
Epoch 8/10, Loss: 2.2322, Accuracy: 77.22%, Time: 20.69s
Epoch 9/10, Loss: 2.0832, Accuracy: 77.96%, Time: 21.32s
Epoch 10/10, Loss: 2.2004, Accuracy: 80.18%, Time: 21.19s
Test Accuracy: 80.86%


In [3]:
# Import necessary libraries
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, models, transforms
import time

# Set device (GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Directories for train and test data
train_dir = '/kaggle/input/freshness1/data/fruits_split/train'
test_dir = '/kaggle/input/freshness1/data/fruits_split/test'

# Define data transformations for train and test sets
train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

test_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Load datasets
train_dataset = datasets.ImageFolder(train_dir, transform=train_transforms)
test_dataset = datasets.ImageFolder(test_dir, transform=test_transforms)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Load the pre-trained VGG16 model
model = models.vgg16(pretrained=True)

# Freeze the feature extraction layers
for param in model.features.parameters():
    param.requires_grad = False

# Modify the classifier part to match our number of classes (e.g., 13 for fruit freshness)
num_classes = len(train_dataset.classes)
model.classifier[6] = nn.Linear(model.classifier[6].in_features, num_classes)

# Move model to the appropriate device
model = model.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.classifier.parameters(), lr=0.001) #use 0.01

# Training function
def train_model(model, criterion, optimizer, train_loader, num_epochs=25): #use 50 epochs
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        correct = 0
        total = 0
        start_time = time.time()
        
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            
            # Zero the parameter gradients
            optimizer.zero_grad()
            
            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            # Backward pass and optimization
            loss.backward()
            optimizer.step()
            
            # Track the loss and accuracy
            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        
        epoch_loss = running_loss / len(train_loader)
        epoch_acc = 100 * correct / total
        elapsed_time = time.time() - start_time
        
        print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.2f}%, Time: {elapsed_time:.2f}s')

# Evaluation function
def evaluate_model(model, test_loader):
    model.eval()  # Set model to evaluation mode
    correct = 0
    total = 0
    with torch.no_grad():  # No need to track gradients for evaluation
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    accuracy = 100 * correct / total
    print(f'Test Accuracy: {accuracy:.2f}%')

# Train the model
train_model(model, criterion, optimizer, train_loader, num_epochs=50)

# Evaluate the model on the test set
evaluate_model(model, test_loader)

# Save the trained model
model_save_path = '/kaggle/working/vgg16_fruit_freshness_model_50.pth'
torch.save(model.state_dict(), model_save_path)
print(f'Model saved to {model_save_path}')


Epoch 1/50, Loss: 2.1141, Accuracy: 54.22%, Time: 21.27s
Epoch 2/50, Loss: 1.4798, Accuracy: 64.20%, Time: 20.87s
Epoch 3/50, Loss: 1.6201, Accuracy: 69.90%, Time: 21.17s
Epoch 4/50, Loss: 1.5066, Accuracy: 75.67%, Time: 20.76s
Epoch 5/50, Loss: 1.7920, Accuracy: 75.30%, Time: 21.23s
Epoch 6/50, Loss: 2.1813, Accuracy: 75.81%, Time: 21.02s
Epoch 7/50, Loss: 2.0401, Accuracy: 78.11%, Time: 21.15s
Epoch 8/50, Loss: 2.5424, Accuracy: 74.93%, Time: 20.84s
Epoch 9/50, Loss: 2.3278, Accuracy: 78.40%, Time: 21.20s
Epoch 10/50, Loss: 1.6653, Accuracy: 80.03%, Time: 20.85s
Epoch 11/50, Loss: 1.6536, Accuracy: 80.77%, Time: 21.17s
Epoch 12/50, Loss: 1.5848, Accuracy: 82.62%, Time: 20.92s
Epoch 13/50, Loss: 1.3598, Accuracy: 84.25%, Time: 21.34s
Epoch 14/50, Loss: 2.1558, Accuracy: 82.54%, Time: 20.95s
Epoch 15/50, Loss: 1.9018, Accuracy: 81.95%, Time: 20.90s
Epoch 16/50, Loss: 1.9365, Accuracy: 81.80%, Time: 21.35s
Epoch 17/50, Loss: 1.7337, Accuracy: 82.10%, Time: 20.81s
Epoch 18/50, Loss: 1.22